In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import plotly.graph_objects as go

In [2]:
usa = gpd.read_file("./../data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")

In [3]:
fuelev = pd.read_csv("../data/Average_retail_price_of_electricity.csv", skiprows=5)

fuelev["State"] = fuelev["Unnamed: 0"].str.slice(3, 5)
# fuelev["price"] = fuelev.iloc[:,-25:-13].mean(axis=1)

In [5]:
melted_data = pd.melt(fuelev, id_vars=['State'], value_vars=[col for col in fuelev.columns if col.isdigit()], var_name='YearMonth', value_name='price')

# Display the first few rows of the melted dataframe to check the transformation
melted_data

,State,YearMonth,price
0,AK,200101,11.22
1,AL,200101,6.33
2,AR,200101,6.88
3,AZ,200101,6.99
4,CA,200101,10.89
...,...,...,...
14122,VT,202401,20.98
14123,WA,202401,11.09
14124,WI,202401,16.46
14125,WV,202401,13.64


In [6]:
plotly_map = usa.merge(melted_data, left_on='STUSPS', right_on='State')
plotly_map = plotly_map[~plotly_map["State"].isin(["AK", "HI"])]

In [8]:
fig = go.Figure()

color_scale = ["#fcffa4", "#bc3754", "#000004"]

unique_months = sorted(melted_data['YearMonth'].unique())
start_month = '202301'

for year in unique_months:
    fig.add_trace(
        go.Choropleth(
            locations=plotly_map['State'].unique(),
            z=plotly_map[plotly_map['YearMonth'] == year]['price'],  # or any other variable
            locationmode='USA-states',
            visible=(year == "202301"),
            colorscale=color_scale,
        )
    )

steps = []
for i, year in enumerate(unique_months):
    step = {
        'method': 'update',
        'args': [{'visible': [False] * len(unique_months)},  # Make all traces invisible
                 {'title': f'Showing data for year: {year}'}],  # Update the title
        'label': str(year)
    }
    step['args'][0]['visible'][i] = True
    steps.append(step)

sliders = [{
    'active': unique_months.index(start_month),
    'currentvalue': {'prefix': 'YearMonth: '},
    'steps': steps
}]

fig.update_layout(
    sliders=sliders,
    title="Electricity Retail Price by State Over Time",
    geo_scope='usa',
)

fig.show()

In [26]:


color_scale = ["#fcffa4", "#bc3754", "#000004"]


fig_map1 = go.Figure(go.Choropleth(
    locations=plotly_map['State'],
    z=plotly_map["price"],
    locationmode='USA-states',
    colorscale=color_scale,
))

fig_map1.update_layout(
    title_text='Electricity retail price (cents/kWh)',
    geo_scope='usa',
    legend_title="Percent of EV registered",
    # showlegend=False,
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",  # Change this to "normal" to fix the order
        title_font_family="Arial",
        font=dict(
            family="Arial",
            size=3,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),

    
)



fig_map1.layout.showlegend = False

fig_map1.show()

KeyError: 'Year'